In [1]:


import numpy as np
from constraint_builder import NonConvexDecomposition
import polytope  

### Velocity constraints
vx_max = 0.05  # Maximum velocity in x-direction (m/s)
vy_max = 0.05  # Maximum velocity in y-direction (m/s)
vel_vertices = np.array([[vx_max, vy_max], [vx_max, -vy_max], [-vx_max, -vy_max], [-vx_max, vy_max]])


#### Acceleration constraints
ax_max = 1.0  # Maximum acceleration in x-direction (m/s^2)
ay_max = 1.0  # Maximum acceleration in y-direction (m/s^2)
accel_vertices = np.array([[ax_max, ay_max], [ax_max, -ay_max], [-ax_max, -ay_max], [-ax_max, ay_max]])


### In X-Y coordinate ###
Workspace = np.array(([[178,  47],
    [704,  47],
    [704, 817],
    [178, 817]]))


## Convex Unsafe Region: Like vertices 
# Obs1 =  [array([[568, 429],
Obs1 = np.array([[568, 429],
                [703, 429],
                [703, 816],
                [568, 816]])

Obs2 = np.array([[308, 181],
                [438, 181],
                [438, 691],
                [308, 691]]) 

Obs3 = np.array([[437,  47],
                [568,  47],
                [568, 309],
                [437, 309]])

# Convex Inequality Constraints: Ax <= b
w_poly = poly = polytope.qhull(Workspace)
Obs = [Obs1, Obs2, Obs3]
obs_poly = [polytope.qhull(box) for box in Obs]
vel_poly = polytope.qhull(vel_vertices)
accel_poly = polytope.qhull(accel_vertices)

wA = w_poly.A
wb = w_poly.b

## velocity convex constraint
## 0, 0 at x-y coordinate only vx, vy is constrained.
F_v = np.hstack([np.zeros((vel_poly.A.shape[0], 2)), vel_poly.A])
f_v = vel_poly.b

# Sate Convex Inequality Constraints: Ax <= b
F_w = np.hstack([wA, np.zeros((wA.shape[0], 2))])
f_w = wb

# Obstacle unsafe region
## For each obstacle polygon convex unsafe set in x-y coor, 0, 0, added for velocity
F_obs = []
for obs in obs_poly:
    F = np.hstack([obs.A, np.zeros((obs.A.shape[0], 2))])
    fvec = obs.b
    F_obs.append((F, fvec))

safe_state_sets = [
(F_w, f_w),
(F_v, f_v),
]

# 2) Unsafe sets (inside of obstacles) as (A, b) with A x <= b
unsafe_state_sets = []
for obs in F_obs:
    unsafe_state_sets.append((np.asarray(obs[0], dtype=float), np.asarray(obs[1], dtype=float)))

# 3) Build facet-wise disjunctions with an optional margin (eps)
nonconvex_decomp = NonConvexDecomposition(eps=0.0)
Fx_list, fx_list = nonconvex_decomp.build(safe_state_sets, unsafe_state_sets)

Gu = accel_poly.A
gu = accel_poly.b

# State transition matrix with damping
dt = 0.1; damping_fun = 1 - 0.1 * dt
A = np.array([[1, 0, dt, 0],
                   [0, 1, 0, dt],
                   [0, 0, damping_fun, 0],
                   [0, 0, 0, damping_fun]])

B = np.array([[0.5 * dt**2, 0],
           [0, 0.5 * dt**2],
           [dt, 0],
           [0, dt]])


E = np.array([[1., 0., 0., 0.],
                           [0., 1., 0., 0.],
                           [0., 0., 1., 0.],
                           [0., 0., 0., 1.]])

Gw = np.array([[ 1.,  0.,  0.,  0.],
                             [ 0.,  1.,  0.,  0.],
                             [ 0.,  0.,  1.,  0.],
                             [ 0.,  0.,  0.,  1.],
                             [-1., -0., -0., -0.],
                             [-0., -1., -0., -0.],
                             [-0., -0., -1., -0.],
                             [-0., -0., -0., -1.]])
        
Fw = np.array([[0.00220696731294516, 0.00033361141907932, 0.01511132840251176, 0.00392036854057386, 0.00137375336326578, 0.00060107734896421, 0.01921057797980761, 0.00721068247405685]]).T
      

In [2]:
import sys, os
from pathlib import Path

sys.path.insert(0, '../')
import config1
cfg = config1.config()

project_root /home/shivam/Aalto Dropbox/Shivam Chaubey/research_work/codes/assistive_system/dev/misc


In [3]:
    sys.path.insert(0,'../')
    from config1 import config
    cfg = config()

    from constraint_builder import ConstraintHelper

    helper = ConstraintHelper(eps=0.0)

    Workspace = np.array([[178, 47], [704, 47], [704, 817], [178, 817]], dtype=float)
    Obs1 = np.array([[568, 429], [703, 429], [703, 816], [568, 816]], dtype=float)
    Obs2 = np.array([[308, 181], [438, 181], [438, 691], [308, 691]], dtype=float)
    Obs3 = np.array([[437,  47], [568,  47], [568, 309], [437, 309]], dtype=float)
    vel_vertices   = np.array([[0.05, 0.05], [0.05, -0.05], [-0.05, -0.05], [-0.05, 0.05]], dtype=float)
    accel_vertices = np.array([[1.0, 1.0], [1.0, -1.0], [-1.0, -1.0], [-1.0, 1.0]], dtype=float)

    Fx_list, fx_list, Gu, gu = helper.construct_state_constraints(
        workspace_vertices=Workspace,
        obstacle_vertices_list=[Obs1, Obs2, Obs3],
        vel_vertices=vel_vertices,
        accel_vertices=accel_vertices,
    )

    E = cfg.E
    Gw = cfg.Gw
    Fw = cfg.Fw

    from CIS_computation import ExtractCIS
    cis_builder = ExtractCIS(cfg, Fx_list = Fx_list, fx_list = fx_list,
                    Gu = Gu, gu = gu, E = E, Gw = Gw, 
                    Fw = Fw, use_matlab = False, safe_set_path = None)

    cis_builder.compute_cis()
    cis_builder.save_cis_npz()
    print("CIS computation finished.")
    print("C_list:", cis_builder.C_list)
    print("c_list:", cis_builder.c_list)


project_root /home/shivam/Aalto Dropbox/Shivam Chaubey/research_work/codes/assistive_system/dev/misc
<<<<<<<<<< matlab engine initialization >>>>>>>>>
matlab engine started: path /home/shivam/Aalto Dropbox/Shivam Chaubey/research_work/codes/assistive_system/dev/misc/lib
> In path (line 109)
In addpath (line 86)
> In path (line 109)
In addpath (line 86)
> In path (line 109)
In addpath (line 86)
matlab engine started: path /home/shivam/Aalto Dropbox/Shivam Chaubey/research_work/codes/assistive_system/dev/misc/lib
> In path (line 109)
In addpath (line 86)
> In path (line 109)
In addpath (line 86)
> In path (line 109)
In addpath (line 86)
matlab engine started: path /home/shivam/Aalto Dropbox/Shivam Chaubey/research_work/codes/assistive_system/dev/misc/lib
> In path (line 109)
In addpath (line 86)
> In path (line 109)
In addpath (line 86)
> In path (line 109)
In addpath (line 86)
unknown error
matlab engine started: path /home/shivam/Aalto Dropbox/Shivam Chaubey/research_work/codes/assisti

AttributeError: 'NoneType' object has no attribute 'genpath'

In [4]:

def load_cis_npz(path):
    """Load CIS .npz and return a handy dict with Python-native types."""
    data = np.load(path, allow_pickle=True)

    out = {
        # compact CIS: nested list of [H|f] arrays (each is shape (m_i, n+1))
        # split forms, if you want them
        "C_list":   data["C_list"].tolist(),   # nested list of H arrays
        "c_list":   data["c_list"].tolist(),   # nested list of f arrays

        # system and constraints
        "A":  data["A"],
        "B":  data["B"],
        "Fx_list": data["Fx_list"].tolist(),
        "fx_list": data["fx_list"].tolist(),
        "Gu": data["Gu"],
        "gu": data["gu"],
        "E":  data["E"],
        "Gw": data["Gw"],
        "Fw": data["Fw"],
        "fw": data["fw"],  # alias

        # config/meta (stored as a Python dict)
        "L": data["L"].item(),
        "T": data["T"].item(),
        "implicit": bool(data["implicit"]),
    }
    return out

In [ ]:
from assistive_control import MIQSingleStep

cfg = config1.config()

saved_CIS = load_cis_npz(cfg.save_cis_path)
A = saved_CIS["A"]
B = saved_CIS["B"]
Fx_list = saved_CIS["Fx_list"]
fx_list = saved_CIS["fx_list"]
Gu = saved_CIS["Gu"]
gu = saved_CIS["gu"]
E = saved_CIS["E"]
Gw = saved_CIS["Gw"]
Fw = saved_CIS["Fw"]
fw = saved_CIS["fw"]

C_list = saved_CIS["C_list"]
c_list = saved_CIS["c_list"]

N = 100
xref = []
uref = [[0.1, 0.1] for i in range(N-1)]
# F = cfg.Fx
# f = cfg.fx
F = None; f = None
# A = cfg.A; B = cfg.B
G = None; g = None
# C_list = cfg.C_list; c_list = cfg.c_list


miqp = MIQSingleStep(A, B, Fxmat = F, fxvec = f, Gumat = G, guvec = g,
T_list=None, t_list=None, C_list=C_list, c_list=c_list,
M=10e8, u_ref=None,
gurobi_params=None, method = "gurobi", warm_start=False)

x0 = np.array([0.695, 0.160, 0.055, 0.055])
miqp.init_prob(x0)

project_root /home/shivam/Aalto Dropbox/Shivam Chaubey/research_work/codes/assistive_system/dev/misc
i, j 0 0
sp.csr_matrix((mC_ij, n)), (12, 4)
i, j 0 1
sp.csr_matrix((mC_ij, n)), (12, 4)
i, j 0 2
sp.csr_matrix((mC_ij, n)), (12, 4)
i, j 0 3
sp.csr_matrix((mC_ij, n)), (12, 4)
i, j 1 0
sp.csr_matrix((mC_ij, n)), (12, 4)
i, j 1 1
sp.csr_matrix((mC_ij, n)), (12, 4)
i, j 1 2
sp.csr_matrix((mC_ij, n)), (12, 4)
i, j 1 3
sp.csr_matrix((mC_ij, n)), (12, 4)
i, j 2 0
sp.csr_matrix((mC_ij, n)), (12, 4)
i, j 2 1
sp.csr_matrix((mC_ij, n)), (12, 4)
i, j 2 2
sp.csr_matrix((mC_ij, n)), (12, 4)


In [6]:
miqp = MIQSingleStep(A, B, Fxmat = F, fxvec = f, Gumat = G, guvec = g,
T_list=None, t_list=None, C_list=cfg.C_list, c_list=cfg.c_list,
M=10e8, u_ref=None,
gurobi_params=None, method = "gurobi", warm_start=False)
miqp.init_prob(x0)

i, j 0 0
sp.csr_matrix((mC_ij, n)), (12, 4)
i, j 0 1
sp.csr_matrix((mC_ij, n)), (12, 4)
i, j 1 0
sp.csr_matrix((mC_ij, n)), (12, 4)
i, j 1 1
sp.csr_matrix((mC_ij, n)), (12, 4)
i, j 1 2
sp.csr_matrix((mC_ij, n)), (12, 4)
i, j 1 3
sp.csr_matrix((mC_ij, n)), (12, 4)
i, j 2 0
sp.csr_matrix((mC_ij, n)), (12, 4)
i, j 2 1
sp.csr_matrix((mC_ij, n)), (12, 4)
i, j 2 2
sp.csr_matrix((mC_ij, n)), (12, 4)


In [ ]:
cfg.C_list[0]

[array([[  99.50249,    0.     ,    5.02488,    0.     ],
        [ -99.50249,    0.     ,   -5.02488,    0.     ],
        [ 140.71603,    0.     ,    0.     ,    0.     ],
        [-140.71603,    0.     ,    0.     ,    0.     ],
        [   0.     , -140.71603,    0.     ,    0.     ],
        [   0.     ,    0.     ,   -7.07107,    0.     ],
        [   0.     ,    0.     ,    0.     ,   -7.07107],
        [   0.     ,  140.71603,    0.     ,    0.     ],
        [   0.     ,    0.     ,    7.07107,    0.     ],
        [   0.     ,    0.     ,    0.     ,    7.07107],
        [   0.     ,   99.50249,    0.     ,    5.02488],
        [   0.     ,  -99.50249,    0.     ,   -5.02488]]),
 array([[  99.50249,    0.     ,    5.02488,    0.     ],
        [ -99.50249,    0.     ,   -5.02488,    0.     ],
        [   0.     , -140.71603,    0.     ,    0.     ],
        [-140.71603,    0.     ,    0.     ,    0.     ],
        [   0.     ,    0.     ,   -7.07107,    0.     ],
        [   

In [ ]:
C_list = saved_CIS["C_list"]
C_list[2][1].shape

(0, 0)

In [ ]:
c_list[0][0]

array([ 56517.05099, -17711.63913,  79926.39217,  -6613.7378 ,
       -25047.64592, 114964.94623,      0.21771,      0.32583,
            0.30257,      0.2467 ,  81293.65913,  -4676.54351])

In [ ]:


T_list = np.vstack([F, Fx])
t_list = np.hstack([fvec, fxvec])


In [ ]:
F

array([-1., -0.,  0.,  0.])